In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import itertools
from collections import Counter
from networkx.drawing.nx_agraph import graphviz_layout
from skbio.stats.composition import ilr
from skbio.stats.composition import clr
from skbio.stats.composition import multiplicative_replacement
import seaborn as sns
from matplotlib import rcParams
sns.set()
sns.set(font_scale=1.5)

In [2]:
globalSpeciesAll = pd.read_csv("./DataFiles/Dataframes/allCohorts_allSpecies.csv")
GoodSampleNames = list(globalSpeciesAll['SampleName'])
print(len(GoodSampleNames))

606


## Import Cohort base pair dict or csvs

In [3]:
IndianBaseDF = pd.read_csv("./DataFiles/CohortInformation/IndianCohortBasePairCount.csv").set_index("SampleName")
IndianBaseDF['CleanName']='temp'
for i in IndianBaseDF.index:
    IndianBaseDF.at[i,'CleanName'] = i.split("_")[0]
IndianBaseDF2 = IndianBaseDF.reset_index().set_index("CleanName").drop(columns=['SampleName']).copy()
IndianBaseDF2.index.name='SampleName'
IndianBaseDF2.head()

,Total_Base_Pairs
SampleName,
SRR5898959,340816565
SRR5898909,370680791
SRR5898960,537013841
SRR5898910,1019643047
SRR5898961,760906549


In [4]:
IndianBaseDF['Total_Base_Pairs'].sum()

63428287648

In [5]:
##############################################################

In [6]:
JapaneseBaseDF = pd.read_csv("./DataFiles/CohortInformation/JapaneseCohortBasePairCount.csv").set_index("SampleName")
JapaneseBaseDF.head()
JapaneseBaseDF['CleanName']='temp'
for i in JapaneseBaseDF.index:
    JapaneseBaseDF.at[i,'CleanName'] = i.split(".")[0]
JapaneseBaseDF2 = JapaneseBaseDF.reset_index().set_index("CleanName").drop(columns=['SampleName']).copy()
JapaneseBaseDF2.index.name='SampleName'
JapaneseBaseDF2.head()

,Total_Base_Pairs
SampleName,
DRR171610,3961598518
DRR171608,3546867040
DRR171613,3802003047
DRR127524,5597591218
DRR171616,3874132490


In [7]:
JapaneseBaseDF['Total_Base_Pairs'].sum()

600931040883

In [8]:
##################################################################

In [25]:
import json

with open("./DataFiles/CohortInformation/American.json", "r") as read_file:
    americanBaseDict = json.load(read_file)

americanBaseDictClean = {'SampleName':[],'Total_Base_Pairs':[]}

#This will say it breaks but its fine.
i=0
sampleNameCheck = 'holder'
while sampleNameCheck != 'bad':
    if 'sub' not in americanBaseDict['SampleName'][i]:
        americanBaseDictClean['Total_Base_Pairs'].append(int(americanBaseDict['Total_Base_Pairs'][i]))
        americanBaseDictClean['SampleName'].append(americanBaseDict['SampleName'][i].split(".")[0])
        i+=1
    else:
        sampleNameCheck='bad'

IndexError: list index out of range

In [26]:
americanBaseDF = pd.DataFrame(data=americanBaseDictClean).set_index("SampleName")
americanBaseDF.head()

,Total_Base_Pairs
SampleName,
SRS015782,4903111306
SRS016095,5020812438
SRS015264,3042360990
SRS015578,5522656290
SRS015190,5488863816


In [24]:
americanBaseDF['Total_Base_Pairs'].sum()

1027510687444

In [13]:
##############################################################################################

In [27]:
import json

with open("/home/mark/Desktop/Research/GlobalNetwork/masterFiles2/Cohorts/baseCounts/European.json", "r") as read_file2:
    europeanBaseDict = json.load(read_file2)
    
europeanBaseDictClean = {'SampleName':[],'Total_Base_Pairs':[]}

i=0
sampleNameCheck = 'holder'
while sampleNameCheck != 'bad':
    if 'sub' not in europeanBaseDict['SampleName'][i]:
        europeanBaseDictClean['Total_Base_Pairs'].append(int(europeanBaseDict['Total_Base_Pairs'][i]))
        europeanBaseDictClean['SampleName'].append(europeanBaseDict['SampleName'][i].split(".")[0])
        i+=1
    else:
        sampleNameCheck='bad'

IndexError: list index out of range

In [28]:
europeanBaseDF = pd.DataFrame(data=europeanBaseDictClean).set_index("SampleName")
europeanBaseDF.head()

,Total_Base_Pairs
SampleName,
ERR011091,594594300
ERR011092,335823825
ERR011093,814222575
ERR011094,739346025
ERR011101,717389325


In [29]:
europeanBaseDF['Total_Base_Pairs'].sum()

45224753550

In [30]:
globalSpeciesAll2 = globalSpeciesAll.set_index("SampleName").copy()

In [31]:
TotalBaseCount = int(europeanBaseDF['Total_Base_Pairs'].sum())+int(americanBaseDF['Total_Base_Pairs'].sum())+int(JapaneseBaseDF['Total_Base_Pairs'].sum())+int(IndianBaseDF['Total_Base_Pairs'].sum())
TotalBaseCount

1737094769525

## Load Small Sample Information DataFrame

In [32]:
allCohortInfoDF = pd.read_csv("./DataFiles/CohortInformation/SampleMappingInformation.csv").set_index("SampleName")
HealthyCohortDF = allCohortInfoDF[(allCohortInfoDF['Cohort']!='nonIBD') & (allCohortInfoDF['Cohort']!='IBD')&(allCohortInfoDF['Cohort']!='CRC') &(allCohortInfoDF['Cohort']!='Chinese')].copy()
HealthyCohortDF.head()

,Cohort,Total_Reads,Mapped,Fraction_Mapped
SampleName,,,,
SRS019068,American,146440506.0,90989903.0,0.621344
SRS015431,American,145342910.0,91257053.0,0.627874
SRS147766,American,124128156.0,77035032.0,0.620609
SRS103987,American,116236100.0,71842871.0,0.618077
SRS143876,American,111457769.0,68106301.0,0.611050


## Average Read Length Per Sample

In [33]:
rlDictionary = {}
with open("./DataFiles/CohortInformation/allCohorts.txt") as rlFile:
    for line in rlFile:
        rlDictionary[line.split(".")[0].split("_R1")[0].split("_TR")[0].split(".")[0].strip()]=line.split("\t")[1].strip()
rlDictionary

{'SRS024132': '90',
 'SRS024265': '95',
 'SRS019582': '96',
 'SRS011061': '95',
 'SRS020233': '93',
 'SRS016517': '76',
 'SRS024331': '91',
 'SRS016753': '93',
 'SRS019601': '93',
 'SRS011084': '92',
 'SRS014979': '94',
 'SRS016954': '91',
 'SRS024075': '93',
 'SRS011134': '90',
 'SRS024435': '87',
 'SRS016989': '90',
 'SRS015264': '96',
 'SRS011239': '92',
 'SRS015578': '94',
 'SRS017103': '95',
 'SRS020328': '97',
 'SRS011271': '91',
 'SRS042284': '94',
 'SRS017191': '97',
 'SRS015369': '85',
 'SRS011302': '91',
 'SRS021484': '96',
 'SRS017247': '93',
 'SRS015431': '83',
 'SRS011405': '94',
 'SRS020869': '92',
 'SRS011452': '95',
 'SRS022609': '88',
 'SRS017307': '96',
 'SRS022137': '88',
 'SRS011529': '92',
 'SRS024388': '93',
 'SRS017433': '95',
 'SRS015663': '96',
 'SRS011586': '90',
 'SRS015794': '93',
 'SRS017521': '93',
 'SRS021948': '96',
 'SRS012273': '90',
 'SRS043701': '92',
 'SRS017701': '92',
 'SRS045528': '92',
 'SRS017821': '90',
 'SRS022071': '96',
 'SRS012902': '94',
